# Video: Structuring Tables for Joins in Pandas

This video describes how to use and manage data frame indexes for efficient joins in pandas.

## Incoming Data to Their Spec

* Often data comes structured differently than what you need...
  * Sometimes you can push for a change upstream.
  * Sometimes you need to handle it as is.

## The Original Garden Bed Materials Data

![Screenshot off Google Sheet showing calculation of material requirements](https://raw.githubusercontent.com/bu-cds-omds/bu-cds-omds-data/main/data/garden-bed_size_materials-original.png)

## Code Example: Reshaping Spreadsheet Data

In [51]:
import pandas as pd

In [52]:
bed_size_materials_original = pd.read_csv("https://raw.githubusercontent.com/bu-cds-omds/bu-cds-omds-data/main/data/garden-bed_size_materials-original.tsv", sep="\t")
bed_size_materials_original

,Bed Width (ft),Bed Length (ft),Height Target (in),Bottom Target (in),Bottom Pieces,Bottom Width (in),Height Pieces,"2"" x 6"" x 4'","2"" x 6"" x 8'","8"" x 8"" x 16"" Cinder Block"
0,4,4,18,45,8,44,3,20,0,12
1,4,8,18,45,8,44,3,6,14,24


In [53]:
bed_size_materials_original["bed_size"] = bed_size_materials_original["Bed Width (ft)"].astype("string") + "' x " + bed_size_materials_original["Bed Length (ft)"].astype('string') + "'"
bed_size_materials_original

,Bed Width (ft),Bed Length (ft),Height Target (in),Bottom Target (in),Bottom Pieces,Bottom Width (in),Height Pieces,"2"" x 6"" x 4'","2"" x 6"" x 8'","8"" x 8"" x 16"" Cinder Block",bed_size
0,4,4,18,45,8,44,3,20,0,12,4' x 4'
1,4,8,18,45,8,44,3,6,14,24,4' x 8'


In [54]:
bed_size_materials = bed_size_materials_original.melt(
    id_vars=["bed_size"],
    value_vars=[c for c in bed_size_materials_original.columns if " x " in c],
    var_name="material",
    value_name="quantity_per_bed")
bed_size_materials

,bed_size,material,quantity_per_bed
0,4' x 4',"2"" x 6"" x 4'",20
1,4' x 8',"2"" x 6"" x 4'",6
2,4' x 4',"2"" x 6"" x 8'",0
3,4' x 8',"2"" x 6"" x 8'",14
4,4' x 4',"8"" x 8"" x 16"" Cinder Block",12
5,4' x 8',"8"" x 8"" x 16"" Cinder Block",24


In [55]:
material_costs = pd.read_csv(
    "https://raw.githubusercontent.com/bu-cds-omds/bu-cds-omds-data/main/data/garden-material_costs.tsv",
    sep="\t",
    index_col="material")
material_costs

,unit_cost
material,
"2"" x 6"" x 4'",4.92
"2"" x 6"" x 8'",6.62
"8"" x 8"" x 16"" Cinder Block",2.53


In [56]:
bed_size_costs = bed_size_materials.join(material_costs, on="material")
bed_size_costs

,bed_size,material,quantity_per_bed,unit_cost
0,4' x 4',"2"" x 6"" x 4'",20,4.92
1,4' x 8',"2"" x 6"" x 4'",6,4.92
2,4' x 4',"2"" x 6"" x 8'",0,6.62
3,4' x 8',"2"" x 6"" x 8'",14,6.62
4,4' x 4',"8"" x 8"" x 16"" Cinder Block",12,2.53
5,4' x 8',"8"" x 8"" x 16"" Cinder Block",24,2.53


In [57]:
bed_size_costs["cost"] = bed_size_costs["quantity_per_bed"] * bed_size_costs["unit_cost"]
bed_size_costs

,bed_size,material,quantity_per_bed,unit_cost,cost
0,4' x 4',"2"" x 6"" x 4'",20,4.92,98.40
1,4' x 8',"2"" x 6"" x 4'",6,4.92,29.52
2,4' x 4',"2"" x 6"" x 8'",0,6.62,0.00
3,4' x 8',"2"" x 6"" x 8'",14,6.62,92.68
4,4' x 4',"8"" x 8"" x 16"" Cinder Block",12,2.53,30.36
5,4' x 8',"8"" x 8"" x 16"" Cinder Block",24,2.53,60.72


In [58]:
bed_size_costs = bed_size_costs.groupby("bed_size")[["cost"]].sum()
bed_size_costs

,cost
bed_size,
4' x 4',128.76
4' x 8',182.92


In [59]:
bed_size_materials.pivot_table(
    index="bed_size",
    columns="material",
    values="quantity_per_bed"
)

material,"2"" x 6"" x 4'","2"" x 6"" x 8'","8"" x 8"" x 16"" Cinder Block"
bed_size,,,
4' x 4',20,0,12
4' x 8',6,14,24
